In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("recommender").getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train_data, test_data = data.randomSplit([0.8,0.2])

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId")

In [10]:
model = als.fit(train_data)

In [11]:
predictions = model.transform(test_data)

In [14]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|   1.0|    26|   1.4923139|
|     31|   1.0|    27|  0.19679809|
|     31|   2.0|    25|    4.407473|
|     85|   1.0|    26|  0.23316789|
|     85|   1.0|    12|  -1.5709426|
|     85|   2.0|    20|   3.7046752|
|     85|   1.0|    29|-0.047326416|
|     85|   3.0|    21|   1.9578538|
|     65|   1.0|    22|  0.18000746|
|     65|   2.0|     3|  0.85863864|
|     65|   2.0|    15|   1.9624407|
|     53|   1.0|     6|   3.1019785|
|     78|   1.0|     1|   1.0590881|
|     78|   1.0|    17|  0.99363786|
|     78|   1.0|     2|   0.9147366|
|     34|   1.0|    28|   1.2848203|
|     34|   3.0|     3|   0.8063948|
|     34|   1.0|    15|   1.3039806|
|     34|   3.0|    25|   3.8792422|
|     28|   1.0|     6| -0.92633444|
+-------+------+------+------------+
only showing top 20 rows



In [15]:
evaluator = RegressionEvaluator(labelCol="rating", metricName="rmse")

In [16]:
evaluator.evaluate(predictions)

1.763585034497697

In [17]:
single_user = test_data.filter(test_data["userId"]==17).select("userId", "movieId")

In [18]:
single_user.show()

+------+-------+
|userId|movieId|
+------+-------+
|    17|      3|
|    17|     10|
|    17|     22|
|    17|     35|
|    17|     45|
|    17|     47|
|    17|     56|
|    17|     73|
|    17|     78|
|    17|     91|
|    17|     93|
+------+-------+



In [20]:
recommendations = model.transform(single_user)

In [21]:
recommendations.orderBy("prediction", ascending=False).show()

+------+-------+-----------+
|userId|movieId| prediction|
+------+-------+-----------+
|    17|     56|  2.7588134|
|    17|     10|  1.8280376|
|    17|     22|  1.7566909|
|    17|     45|   1.624634|
|    17|     47|  1.6128845|
|    17|     93|  1.5270202|
|    17|     91|  1.2672998|
|    17|     73|   1.133004|
|    17|     78| 0.99363786|
|    17|      3|  0.2866574|
|    17|     35|-0.37545285|
+------+-------+-----------+



In [22]:
meal = spark.read.csv("Meal_Info.csv", inferSchema=True, header=True)

In [23]:
meal.show()

+-------+------+------+--------+--------------------+
|movieId|rating|userId|mealskew|           meal_name|
+-------+------+------+--------+--------------------+
|      2|   3.0|     0|     2.0|       Chicken Curry|
|      3|   1.0|     0|     3.0|Spicy Chicken Nug...|
|      5|   2.0|     0|     5.0|           Hamburger|
|      9|   4.0|     0|     9.0|       Taco Surprise|
|     11|   1.0|     0|    11.0|            Meatloaf|
|     12|   2.0|     0|    12.0|        Ceaser Salad|
|     15|   1.0|     0|    15.0|            BBQ Ribs|
|     17|   1.0|     0|    17.0|         Sushi Plate|
|     19|   1.0|     0|    19.0|Cheesesteak Sandw...|
|     21|   1.0|     0|    21.0|             Lasagna|
|     23|   1.0|     0|    23.0|      Orange Chicken|
|     26|   3.0|     0|    26.0|    Spicy Beef Plate|
|     27|   1.0|     0|    27.0|Salmon with Mashe...|
|     28|   1.0|     0|    28.0| Penne Tomatoe Pasta|
|     29|   1.0|     0|    29.0|        Pork Sliders|
|     30|   1.0|     0|    3